In [8]:
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.svm import SVC



In [2]:
dataset_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'datasets')

##### Datos a usar

- V2 --> dataset limpio CAPAZ DESPUÉS
- V5 --> eliminación de las variables cn poca información


##### Pipeline

1. cargar dataset
2. reemplazar NS/NC en convive por NA
3. hacer modificaciones para svm. Considerar: Apply scikit's OneHotEncoder with the handle_unknown parameter set to "ignore"
4. separar NA de convive, serán el dataset "no visto"
5. separar features de target
6. separar train-test NO DEBERÍA HACER LAS MODIFICACIONES EN TRAIN Y TEST POR SEPARADO?
7. armar modelo
8. evaluar


##### Modificaiones para SVM

1. pasar datetime a timestamp y escalar --> OK
2. escalar la variable edad --> OK
3. pasar edad a categórica (en una columna distinta) --> OK
4. pasar todas las categóricas a dummy:
   1. edad, momento del día y estación del año con un encoder ordinal
   2. escalar las features encodeadas con ordinal
   3. variables de violencia SI/NO para 1 y 0
   4. el resto con one hot encoder
5. borrar las categóricas para hacer más pequeño el dataset




In [13]:
# 1. cargar datasets

#llamados_v2= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v2.xlsx'), parse_dates=['llamado_fecha_hora'])
#llamados_v4= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v4.xlsx'), parse_dates=['llamado_fecha_hora'])
llamados_v5= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v5.xlsx'), parse_dates=['llamado_fecha_hora'])


In [14]:
#todos_datasets = [llamados_v2, llamados_v4, llamados_v5]
#datasets_45 = [llamados_v4, llamados_v5]

In [14]:
# 2 reemplazar todos los NS/NC en convive por na

llamados_v5.loc[:, 'victima_convive_agresor'] = llamados_v5['victima_convive_agresor'].replace({'NS/NC': pd.NA})

#for dataset in todos_datasets:

    # Replace the specified values with NaN
#    dataset.loc[:, 'victima_convive_agresor'] = dataset['victima_convive_agresor'].replace({'NS/NC': pd.NA})

3. Modificaciones para SVM



In [15]:
# 1. timestamp-fecha_hora encodeado y escalado
scaler = MinMaxScaler()
encoder = OrdinalEncoder()


llamados_v5['timestamp_encoded'] = encoder.fit_transform(llamados_v5[['llamado_fecha_hora']])
llamados_v5['timestamp_encoded_scaled'] = scaler.fit_transform(llamados_v5[['timestamp_encoded']])
    
llamados_v5.timestamp_encoded_scaled

0        0.000000
1        0.000060
2        0.000121
3        0.000181
4        0.000181
           ...   
19138    0.999758
19139    0.999940
19140    0.999819
19141    0.999879
19142    0.999879
Name: timestamp_encoded_scaled, Length: 19143, dtype: float64

In [18]:
# 2. Edad. A edad le hago dos cosas para dos pruebas distintas: 
# cosa 1. escalo los casos completos.
# cosa 2: la paso a categórica, ahí los faltantes quedan codificados como una categoría más, y ahí luego la encodeo como 
# variable categórica con un ordina encoder para preservar el orden. 



# Cosa 1: edad escalada para pruebas con casos completos de edad
# printear una edad faltante a ver qué hace con esos faltantes
llamados_v5['victima_edad_escalada'] = scaler.fit_transform(llamados_v5[['victima_edad']])
llamados_v5['llamante_edad_escalada'] = scaler.fit_transform(llamados_v5[['llamante_edad']])



# Cosa 2: pasar edad a categórica

def categoria_edad (x):
    if (x >= 0) and (x <= 11) :
        return 'Niñez'
    elif (x >= 12) and (x <=18):
        return 'Adolescencia'
    elif (x >= 19) and (x <=30):
        return 'Juventud'
    elif (x>=31) and (x<=65) :
        return 'Vejez'
    elif x>=66:
        return 'Vejez mayor'
    else:
        return 'NS/NC'



llamados_v5['victima_edad_cat'] = \
llamados_v5.victima_edad.apply(categoria_edad)
llamados_v5['llamante_edad_cat'] = \
llamados_v5.llamante_edad.apply(categoria_edad)
#llamados_v5.drop(['victima_edad','llamante_edad'], axis=1, inplace=True)


# sobra: victima_edad y llamante_edad    


In [21]:
llamados_v5.llamante_edad_escalada

0        0.281250
1             NaN
2        0.458333
3             NaN
4             NaN
           ...   
19138         NaN
19139    0.208333
19140    0.552083
19141    0.385417
19142         NaN
Name: llamante_edad_escalada, Length: 19143, dtype: float64

In [ ]:
# 4. Variables categóricas que serán encodeadas con ordinal: edad cat (ordinal), momento día (ordinal), 
# estación del año (ordinal).
# Además de encodearlas con ordinal, luego voy a escalar ese encoding ordinal. 
# FUENTE Encoding_Methods_for_Categorical_Data.pdf

# 4.1 pasar edad_cat, momento_dia (V5) y estacion_del_año (V5) a dummy con un encoder ordinal

hot_encoder = OneHotEncoder()
    

llamados_v5['victima_edad_cat_encoded'] = encoder.fit_transform(llamados_v5['victima_edad_cat'])
llamados_v5['llamante_edad_cat_encoded'] = encoder.fit_transform(llamados_v5['llamante_edad_cat'])
#llamados_v5.drop(['victima_edad_cat', 'llamante_edad_cat'], axis=1, inplace=True)

#sobra victima_edad_cat y llamante_edad_cat

llamados_v5['momento_dummy'] = encoder.fit_transform(llamados_v5['momento_dia'])
llamados_v5['estacion_dummy'] = encoder.fit_transform(llamados_v5['estacion_del_año'])
#llamados_v5.drop(['estacion_del_año','momento_dia',], axis=1, inplace=True)

#sobra momento_dia y estacion_del_año

# 4.2 escalar las variables ordinales edad, momento del día y estación


llamados_v5['victima_edad_dummy_scaled'] = scaler.fit_transform(llamados_v5[['victima_edad_cat_dummy']])
llamados_v5['llamante_edad_dummy_scaled'] = scaler.fit_transform(llamados_v5[['llamante_edad_cat_dummy']])
llamados_v5['momento_dummy_scaled'] = scaler.fit_transform(llamados_v5[['momento_dummy']])
llamados_v5['estacion_dummy_scaled'] = scaler.fit_transform(llamados_v5[['estacion_dummy']])

#llamados_v5.drop('momento_dummy', axis=1, inplace=True)
#llamados_v5.drop('estacion_dummy', axis=1, inplace=True )

#llamados_v5.drop('victima_edad_cat_dummy', axis=1, inplace=True)
#llamados_v5.drop('llamante_edad_cat_dummy', axis=1, inplace=True )


#sobra victima_edad_cat_dummy y llamante_edad_cat_dummy, momento_dummy y estacion_dummy




# algunas de mis features tienen cardinalidad altísima, el one hot no va a andar bien para esas y el 
# ordinal aunque anda bien introduce un sesgo de ordinalidad que no es verdadero para esas feature.
# una posibilidad es ver qué tan relacionadas están esas features con la variable target y quizás 
# las puedo matar. 


# hacer for loop para encodera el resto de las categóricas TODAS MENOS TARGET
# luego separo el test final que son los vacíos de target
# luego encodeo target 
# label encoder es ordinal, one hot no. Sin embargo, one hot puede ser problemático con alta 
# feature cardinality. Chequeemos feature cardinality

In [39]:
llamados_v5.select_dtypes(include=['object']).nunique()

llamado_provincia                            25
llamante_genero                               4
llamante_vinculo                             16
caso_judicializado                            3
hecho_lugar                                  17
victima_a_resguardo                           2
victima_genero                                4
victima_nacionalidad                          9
victima_vinculo_agresor                      15
victima_discapacidad                          3
victima_convive_agresor                       2
vs_tocamiento_sexual                          2
vs_intento_tocamiento                         2
vs_grooming                                   2
vs_exhibicionismo                             2
vs_obligacion_sacarse_fotos_pornograficas     2
vs_acoso_sexual                               2
vs_iniciacion_sexual_forzada_inducida         2
vs_otra_forma_violencia_sexual                2
vs_no_sabe_no_contesta                        2
ofv_sentimiento_amenaza                 

Si bien al principio la idea era que la primera prueba de svm fuera con el dataset completo, normalizado pero con poca o ninguna intervención en la construcción de variables; llegados a este punto, la cardinalidad de alta de estasvaraibles lleva a tomar la decisón de reducrilas sin antes correr el experimento con svm porque ya está probado en la literatura que alta cardinalidad con encoders tipo one hot es mala y el target o ordinal encoder que funcionan bien para alta cardinalidad no me convencve para etsas vaiables porque no hay ordinalidad que preserar y porque el target implica tener otros cuidados para no incurrir en data leackage


In [ ]:
# Encode features
hot_encoder = OneHotEncoder()
X_encoded = hot_encoder.fit_transform(X).toarray()

# Encode target variable
hot_encoder = OneHotEncoder()
y_encoded = hot_encoder.fit_transform(y.values.reshape(-1, 1)).toarray()

In [ ]:
# sacar todos los casos vacíos de convive, esos serán el dataset aparte final a predecir con el mejor modelo
# encodear el resto del dataset, todo menos y_convive

def one_hot_encoder(dataset):
    columna_excluir = 'victima_convive_agresor'
    columnas_cat = dataset.select_dtypes(include=['object']).columns.tolist()
    columnas_cat.remove(columna_excluir)

    # Apply one-hot encoding to object columns excluding the chosen one
    df_encoded = pd.get_dummies(df, columns=object_columns)

    # Display the DataFrame with one-hot encoded columns
    print(df_encoded)


In [36]:
# 4 separar los na de convive en los nuevos datasets
# donde prueba 1: dataset completo, limpio. Prueba 2: dataset con feature engineering. 
# Prueba 3: dataset con eliminación de variables poco informativas

prueba_1_no_visto = llamados_v2[llamados_v2['victima_convive_agresor'].isna()].copy(deep=True)
prueba_1_train_test = llamados_v2[~llamados_v2['victima_convive_agresor'].isna()].copy(deep=True)

prueba_2_no_visto = llamados_v4[llamados_v4['victima_convive_agresor'].isna()].copy(deep=True)
prueba_2_train_test = llamados_v4[~llamados_v4['victima_convive_agresor'].isna()].copy(deep=True)

prueba_3_no_visto = llamados_v5[llamados_v5['victima_convive_agresor'].isna()].copy(deep=True)
prueba_3_train_test = llamados_v5[~llamados_v5['victima_convive_agresor'].isna()].copy(deep=True)



In [43]:
prueba_1_no_visto.to_excel(os.path.join(dataset_dir, 'xlsx/prueba_1_no_visto.xlsx'), index=False)
prueba_2_no_visto.to_excel(os.path.join(dataset_dir, 'xlsx/prueba_2_no_visto.xlsx'), index=False)
prueba_3_no_visto.to_excel(os.path.join(dataset_dir, 'xlsx/prueba_3_no_visto.xlsx'), index=False)


In [41]:
del llamados_v2, llamados_v4, llamados_v5, prueba_1_no_visto, prueba_2_no_visto, prueba_3_no_visto

In [ ]:
# 5 separar features de target


# Separate features and target
# X

prueba_1_X = prueba_1_train_test.drop('victima_convive_agresor', axis=1)
prueba_2_X = prueba_2_train_test.drop('victima_convive_agresor', axis=1)
prueba_3_X = prueba_3_train_test.drop('victima_convive_agresor', axis=1)

# Y

prueba_1_y = prueba_1_train_test['victima_convive_agresor']
prueba_2_y = prueba_2_train_test['victima_convive_agresor']
prueba_3_y = prueba_3_train_test['victima_convive_agresor']




In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Train SVM model
svm_model = SVC()
svm_model.fit(X_train, y_train)

# Make predictions
predictions = svm_model.predict(X_test)

### Experimentos usando los datasets preparados para svm:

- las variaciones son en las columnas de edad, en los datasets que tengo, y en los kernels de SVM

##### Prueba 1, dataset V2, V3, V4
- edad categórica pasada a dummy se va
- edad numérica se queda pero solo los datos completos de ambas
- corro SVN c =/= kernels

##### Prueba 2, dataset V2, V3, V4
- edad categórica pasada a dummy se va
- edad numérica se queda pero solo los datos completos de victima
- corro SVN c =/= kernels

##### Prueba 3, dataset V2, V3, V4
- edad categórica pasada a dummy se va
- edad numérica se queda pero solo los datos completos de llamante
- corro SVN c =/= kernels

##### Prueba 4, dataset V2, V3, V4
- edad numérica se va
- edad categórica pasada a dummy se queda
- corro SVN c =/= kernels